In [1]:
# Import Packages
import numpy as np
import cv2
from tensorflow import keras
import h5py

In [2]:
def FindFaces_DNN(img, model):
    faces = []
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    model.setInput(blob)
    detections = model.forward()
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > .75:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            faces.append(box.astype("int"))
    return faces

In [3]:
def FindFaces_HCC(img,model):
    faces = []
    detections = model.detectMultiScale(img,scaleFactor=1.1,minNeighbors=3)
    for detection in detections:
        xmin = int(detection[0])
        ymin = int(detection[1])
        width = int(detection[2])
        height = int(detection[3])
        xmax = xmin+width
        ymax = ymin+height
        faces.append((xmin,ymin,xmax,ymax))
    return faces

In [14]:
def ClassifyFace_CNN(img, model):
    resized = cv2.resize(img,(32,32))
    scaledInput = resized/255.0
    test = np.stack([scaledInput])
    prediction = model.predict(test)
    return prediction[0][0]<0.3

In [11]:
# Mask Detection Function
def MaskDetectionVideo(img,findFaces,detector,classifyFace,classifier):
    for (xmin, ymin, xmax, ymax) in findFaces(img,detector):
        face = img[ymin:ymax, xmin:xmax]
        prediction = classifyFace(face,classifier)
        color = [0,0,255] if prediction else [0,255,0]
        cv2.rectangle(img,(xmin,ymin),(xmax,ymax),color,2)
        text = "PLEASE PUT YOUR MASK ON!" if prediction else "Thank you for wearing a mask!"
        y = ymin - 10 if ymin - 10 > 10 else ymin + 10
        cv2.putText(img, text, (xmin, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    return img

In [15]:
# Real-time Demo

## Deep Neural Network Detection
FindFaces = FindFaces_DNN
detector = cv2.dnn.readNetFromCaffe("./Data/models/deploy.prototxt.txt", "./Data/models/classifier.caffemodel")

## Haar Cascade Classifier Detection - Fast but detects many false positives
# FindFaces = FindFaces_HCC
# detector = cv2.CascadeClassifier("./Data/models/haarcascade_frontalface_default.xml")  

# Custom Convolutional Neural Network Classification
ClassifyFaces = ClassifyFace_CNN
classifier=keras.models.load_model("./Data/models/model.h5")

vid = cv2.VideoCapture(0) 
while(True): 
    ret, frame = vid.read() 
    classified = MaskDetectionVideo(frame,FindFaces,detector,ClassifyFaces,classifier)
    cv2.imshow('frame', classified)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
vid.release() 
cv2.destroyAllWindows() 